In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import librosa
import matplotlib.pyplot as plt
import soundfile as sf
from sklearn.model_selection import train_test_split
import tensorflow_addons as tfa


C:\Users\HP\AppData\Roaming\Python\Python311\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
C:\Users\HP\AppData\Roaming\Python\Python311\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.18.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an is

ModuleNotFoundError: No module named 'keras.src.engine'

In [9]:
import librosa
import numpy as np
import os
import cv2  # OpenCV for resizing

def load_audio_files(directory):
    data, labels = [], []
    for label, folder in enumerate(os.listdir(directory)):
        folder_path = os.path.join(directory, folder)
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            signal, sr = librosa.load(file_path, sr=22050)
            
            # Generate Mel spectrogram with 128 Mel bands
            mel_spec = librosa.feature.melspectrogram(y=signal, sr=sr, n_mels=128)
            mel_spec = librosa.power_to_db(mel_spec, ref=np.max)
            
            # Resize to (128, 128)
            mel_spec_resized = cv2.resize(mel_spec, (128, 128))

            # Expand dimensions to add channel (for CNN input compatibility)
            data.append(mel_spec_resized)
            labels.append(label)

    # Convert to numpy arrays and add channel dimension
    data = np.array(data)
    data = np.expand_dims(data, -1)  # Shape will be (num_samples, 128, 128, 1)
    labels = np.array(labels)
    
    return data, labels

# Load the audio data
data, labels = load_audio_files('C:/Users/HP/Downloads/archive/Raw Audio/')

# Verify the shape
print(data.shape)  # Should be (num_samples, 128, 128, 1)


(1586, 128, 128, 1)


In [10]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

latent_dim = 32  # Adjust as needed based on audio data complexity
input_shape = (128, 128, 1)  # Example shape for Mel spectrogram

# Custom Sampling layer for the reparameterization trick
class Sampling(layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

# Encoder for VAE
def build_encoder(input_shape, latent_dim):
    inputs = keras.Input(shape=input_shape)
    x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(inputs)
    x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
    x = layers.Flatten()(x)
    x = layers.Dense(160, activation="relu")(x)
    z_mean = layers.Dense(latent_dim, name="z_mean")(x)
    z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
    z = Sampling()([z_mean, z_log_var])  # Use the Sampling layer here
    return keras.Model(inputs, [z_mean, z_log_var, z], name="encoder")

# Decoder for VAE
def build_decoder(latent_dim, input_shape):
    latent_inputs = keras.Input(shape=(latent_dim,))
    x = layers.Dense(8 * 8 * 64, activation="relu")(latent_inputs)
    x = layers.Reshape((8, 8, 64))(x)
    x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
    x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
    outputs = layers.Conv2DTranspose(1, 3, activation="sigmoid", padding="same")(x)
    return keras.Model(latent_inputs, outputs, name="decoder")

# Discriminator for GAN
def build_discriminator(input_shape):
    inputs = keras.Input(shape=input_shape)
    x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(inputs)
    x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(x)
    x = layers.Flatten()(x)
    x = layers.Dense(64, activation="relu")(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    return keras.Model(inputs, outputs, name="discriminator")

# Instantiate the models
encoder = build_encoder(input_shape, latent_dim)
decoder = build_decoder(latent_dim, input_shape)
discriminator = build_discriminator(input_shape)

# Display model summaries to verify
encoder.summary()
decoder.summary()
discriminator.summary()


Model: "encoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)    │ (None, 128, 128, 1)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_4 (Conv2D)             │ (None, 64, 64, 32)        │             320 │ input_layer_3[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_5 (Conv2D)             │ (None, 32, 32, 64)        │          18,496 │ conv2d_4[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ flatten_2 (Flatten)           │ (None, 65536)             │               0 │ conv2d_5[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_4 (Dense)               │ (None, 160)               │      10,485,920 │ flatten_2[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ z_mean (Dense)                │ (None, 32)                │           5,152 │ dense_4[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ z_log_var (Dense)             │ (None, 32)                │           5,152 │ dense_4[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ sampling_1 (Sampling)         │ (None, 32)                │               0 │ z_mean[0][0],              │
│                               │                           │                 │ z_log_var[0][0]            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 10,515,040 (40.11 MB)

 Trainable params: 10,515,040 (40.11 MB)

 Non-trainable params: 0 (0.00 B)

...

Model: "discriminator"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)           │ (None, 128, 128, 1)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 64, 64, 64)          │             640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 32, 32, 32)          │          18,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 32768)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 64)                  │       2,097,216 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,116,385 (8.07 MB)

 Trainable params: 2,116,385 (8.07 MB)

 Non-trainable params: 0 (0.00 B)

In [20]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow import keras

# Define encoder with (32, 32, 1) input shape
def build_encoder(latent_dim):
    encoder_inputs = keras.Input(shape=(32, 32, 1))
    x = layers.Conv2D(32, (3, 3), activation="relu", strides=2, padding="same")(encoder_inputs)
    x = layers.Conv2D(64, (3, 3), activation="relu", strides=2, padding="same")(x)
    x = layers.Flatten()(x)
    x = layers.Dense(16, activation="relu")(x)
    z_mean = layers.Dense(latent_dim, name="z_mean")(x)
    z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
    
    def sampling(args):
        z_mean, z_log_var = args
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon
    
    z = layers.Lambda(sampling, output_shape=(latent_dim,), name="z")([z_mean, z_log_var])
    return keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")

# Define decoder that takes in latent vector and outputs (32, 32, 1)
def build_decoder(latent_dim):
    latent_inputs = keras.Input(shape=(latent_dim,))
    x = layers.Dense(8 * 8 * 64, activation="relu")(latent_inputs)
    x = layers.Reshape((8, 8, 64))(x)
    x = layers.Conv2DTranspose(64, (3, 3), activation="relu", strides=2, padding="same")(x)
    x = layers.Conv2DTranspose(32, (3, 3), activation="relu", strides=2, padding="same")(x)
    x = layers.Conv2DTranspose(1, (3, 3), activation="sigmoid", padding="same")(x)
    return keras.Model(latent_inputs, x, name="decoder")

# Define discriminator that takes (32, 32, 1) as input
def build_discriminator():
    discriminator_inputs = keras.Input(shape=(32, 32, 1))
    x = layers.Conv2D(64, (3, 3), activation="relu", strides=2, padding="same")(discriminator_inputs)
    x = layers.Conv2D(128, (3, 3), activation="relu", strides=2, padding="same")(x)
    x = layers.Flatten()(x)
    x = layers.Dense(64, activation="relu")(x)
    discriminator_outputs = layers.Dense(1)(x)
    return keras.Model(discriminator_inputs, discriminator_outputs, name="discriminator")

# Instantiate encoder, decoder, and discriminator with desired latent dimension
latent_dim = 16
encoder = build_encoder(latent_dim)
decoder = build_decoder(latent_dim)
discriminator = build_discriminator()


In [22]:
class VAE_GAN(tf.keras.Model):
    def __init__(self, encoder, decoder, discriminator, **kwargs):
        super(VAE_GAN, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.discriminator = discriminator

    def compile(self, vae_optimizer, disc_optimizer, gen_optimizer, **kwargs):
        super(VAE_GAN, self).compile(**kwargs)
        self.vae_optimizer = vae_optimizer
        self.disc_optimizer = disc_optimizer
        self.gen_optimizer = gen_optimizer
        self.reconstruction_loss_fn = keras.losses.MeanSquaredError()
        self.kl_loss_fn = keras.losses.KLDivergence()
        self.gan_loss_fn = keras.losses.BinaryCrossentropy(from_logits=True)

    def train_step(self, data):
        batch_size = tf.shape(data)[0]  # Use tf.shape to get the dynamic batch size

        with tf.GradientTape(persistent=True) as tape:
            # Forward pass
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)

            # Compute reconstruction and KL loss
            reconstruction_loss = tf.reduce_mean(self.reconstruction_loss_fn(data, reconstruction))
            kl_loss = -0.5 * tf.reduce_sum(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            vae_loss = reconstruction_loss + kl_loss

            # Discriminator
            real_labels = tf.ones((batch_size, 1))
            fake_labels = tf.zeros((batch_size, 1))
            disc_loss_real = self.gan_loss_fn(real_labels, self.discriminator(data))
            disc_loss_fake = self.gan_loss_fn(fake_labels, self.discriminator(reconstruction))
            disc_loss = (disc_loss_real + disc_loss_fake) / 2

            # Generator loss
            gen_loss = self.gan_loss_fn(real_labels, self.discriminator(reconstruction))

        # Backpropagation
        vae_gradients = tape.gradient(vae_loss, self.encoder.trainable_weights + self.decoder.trainable_weights)
        disc_gradients = tape.gradient(disc_loss, self.discriminator.trainable_weights)
        gen_gradients = tape.gradient(gen_loss, self.decoder.trainable_weights)

        self.vae_optimizer.apply_gradients(zip(vae_gradients, self.encoder.trainable_weights + self.decoder.trainable_weights))
        self.disc_optimizer.apply_gradients(zip(disc_gradients, self.discriminator.trainable_weights))
        self.gen_optimizer.apply_gradients(zip(gen_gradients, self.decoder.trainable_weights))

        return {
            "vae_loss": vae_loss,
            "disc_loss": disc_loss,
            "gen_loss": gen_loss,
            "reconstruction_loss": reconstruction_loss,
            "kl_loss": kl_loss,
        }

# Instantiate the model and compile it
vae_gan = VAE_GAN(encoder, decoder, discriminator)
vae_gan.compile(
    vae_optimizer=keras.optimizers.Adam(),
    disc_optimizer=keras.optimizers.Adam(),
    gen_optimizer=keras.optimizers.Adam()
)

# Train the model on resized data
history = vae_gan.fit(train_data_resized, epochs=50, batch_size=32)


Epoch 1/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 9s 27ms/step - disc_loss: 0.0981 - gen_loss: 6.1365 - kl_loss: 27.3046 - reconstruction_loss: 146.1416 - vae_loss: 173.4462
Epoch 2/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - disc_loss: 0.0211 - gen_loss: 6.0726 - kl_loss: 0.0119 - reconstruction_loss: 142.8111 - vae_loss: 142.8230
Epoch 3/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - disc_loss: 0.0178 - gen_loss: 5.2605 - kl_loss: 0.0055 - reconstruction_loss: 144.1577 - vae_loss: 144.1632
Epoch 4/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - disc_loss: 0.0191 - gen_loss: 5.5459 - kl_loss: 0.0029 - reconstruction_loss: 142.3176 - vae_loss: 142.3204
Epoch 5/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - disc_loss: 0.0173 - gen_loss: 5.5771 - kl_loss: 0.0017 - reconstruction_loss: 142.2944 - vae_loss: 142.2961
Epoch 6/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - disc_loss: 0.0176 - gen_loss: 4.9976 - kl_loss: 0.0010 - reconstruction_loss: 144.6220 - vae_loss: 144.6230
Epoch 7/50
40/40 ━━━━━━━━━━━━━━━━

In [40]:
# Define a function to calculate reconstruction errors on test data
def calculate_reconstruction_error(data, model):
    # Encode and decode the data using the trained VAE-GAN model
    z_mean, z_log_var, z = model.encoder(data)
    reconstructed_data = model.decoder(z)

    # Compute reconstruction error (Mean Squared Error)
    reconstruction_error = tf.reduce_mean(tf.square(data - reconstructed_data), axis=[1, 2, 3])
    return reconstruction_error.numpy()


In [41]:
import cv2
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Ensure resizing function works correctly
def resize_data(data, target_shape=(32, 32)):
    resized_data = []
    for img in data:
        img_resized = cv2.resize(img, target_shape)  # Resize to target shape (32, 32)
        resized_data.append(img_resized)
    resized_data = np.array(resized_data)
    resized_data = np.expand_dims(resized_data, -1)  # Add channel dimension for (num_samples, 32, 32, 1)
    return resized_data

# Load and resize test data
test_data, test_labels = load_audio_files('C:/Users/HP/Desktop/Test Data/')  # Load test data
test_data_resized = resize_data(test_data, target_shape=(32, 32))  # Resize to (32, 32, 1)

# Verify the shape of test_data_resized
print("Resized test data shape:", test_data_resized.shape)  # Should print (num_samples, 32, 32, 1)

# Calculate reconstruction error on resized test data
reconstruction_errors = calculate_reconstruction_error(test_data_resized, vae_gan)

# Define a threshold for anomaly detection
threshold = np.percentile(reconstruction_errors, 95)  # Set threshold based on the 95th percentile

# Predict anomalies
anomaly_predictions = reconstruction_errors > threshold

# Evaluation metrics
binary_test_labels = (test_labels == 1)  # 1 for anomaly, 0 for normal
accuracy = accuracy_score(binary_test_labels, anomaly_predictions)
precision = precision_score(binary_test_labels, anomaly_predictions)
recall = recall_score(binary_test_labels, anomaly_predictions)
f1 = f1_score(binary_test_labels, anomaly_predictions)
conf_matrix = confusion_matrix(binary_test_labels, anomaly_predictions)

# Print evaluation results
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:\n", conf_matrix)


Resized test data shape: (92, 32, 32, 1)
Accuracy: 0.9456521739130435
Precision: 0.0
Recall: 0.0
F1 Score: 0.0
Confusion Matrix:
 [[87  5]
 [ 0  0]]


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
